# Creating, Distributing, and Testing Julia Packages with Binary Dependencies

JuliaCon Workshop, June 21 2016

Tony Kelman (`@tkelman`)

- So you wrote or want to use a Julia package that [calls into C or Fortran](http://docs.julialang.org/en/release-0.4/manual/calling-c-and-fortran-code/).
- How do you go from "`ccall` works on my machine" to making your package easy to install for anyone?
- You want `Pkg.add("MyPackage")` to find, build, or download a copy of the library that will work on:
  - Linux (many distributions, Ubuntu, CentOS, etc) on `x86_64` architecture
  - Mac OS X
  - Windows (7,8,10, with 64 bit and 32 bit Julia)
  - More exotic OS or architecture combinations (won't cover in much detail)
- Ideally without requiring the user to have root/admin permissions

In [1]:
# Example library - Rmath, let's fix
# https://github.com/JuliaLang/julia/issues/6131
include("clone_example.jl")

Cloning into 'Rmath-julia'...
INFO: Initializing package repository /home/juser/.julia/v0.4
INFO: Cloning METADATA from git://github.com/JuliaLang/METADATA.jl
INFO: Cloning Rmath from https://github.com/tkelman/Rmath.jl
INFO: Computing changes...
INFO: Cloning cache of BinDeps from git://github.com/JuliaLang/BinDeps.jl.git
INFO: Cloning cache of Compat from git://github.com/JuliaLang/Compat.jl.git
INFO: Cloning cache of SHA from git://github.com/staticfloat/SHA.jl.git
INFO: Cloning cache of URIParser from git://github.com/JuliaWeb/URIParser.jl.git
INFO: Installing BinDeps v0.3.21
INFO: Installing Compat v0.8.1
INFO: Installing SHA v0.1.2
INFO: Installing URIParser v0.1.3
INFO: Package database updated


In [2]:
# If you run this on Windows, it likely won't work -
# See https://github.com/tkelman/JuliaCon2015/blob/master/KelmanJuliaCon2015.pdf
run(`make -C ../Rmath-julia clean`)
run(`make -C ../Rmath-julia -j4`)

make: Entering directory `/home/juser/Rmath-julia'
make -C src clean
make[1]: Entering directory `/home/juser/Rmath-julia/src'
rm -f *.o *.do *.a *.so core* *~ *#
make[1]: Leaving directory `/home/juser/Rmath-julia/src'
make: Leaving directory `/home/juser/Rmath-julia'
make: Entering directory `/home/juser/Rmath-julia'
make -C src
make[1]: Entering directory `/home/juser/Rmath-julia/src'
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c bd0.c -o bd0.o
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c dnbeta.c -o dnbeta.o
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c i1mach.c -o i1mach.o
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c pnbinom.c -o pnbinom.o
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c qlnorm.c -o qlnorm.o
gcc  -Wall -O3 -fPIC -DMATHLIB_STANDALONE   -std=gnu99 -I../include -DNDEBUG -c rgeom.c -o rgeom.o
gcc  -Wal

In [3]:
# Does it work?
if VERSION <= v"0.5"
    # The version that came with Julia
    ccall((:pgamma,"libRmath-julia"), Cdouble,
        (Cdouble,Cdouble,Cdouble,Cint,Cint), 1.0,1.0,1.0,0,0)
end

0.36787944117144233

In [4]:
# The version we just compiled
# Copy it to a new name just to be sure we aren't using
# the same copy we just called
cp("../Rmath-julia/src/libRmath-julia.$(Libdl.dlext)",
    "../Rmath-julia/src/libRmath-julia2.$(Libdl.dlext)")
ccall((:pgamma,"../Rmath-julia/src/libRmath-julia2"), Cdouble,
    (Cdouble,Cdouble,Cdouble,Cint,Cint), 1.0,1.0,1.0,0,0)

0.36787944117144233

Now onto the (other) complicated part!
* Upload the binary somewhere
  * GitHub releases, simplest choice
  * Automation using Travis CI (if we need to change or rebuild often)
* Repeat for other common platforms
* OS-specific caveats (there are many)
* Get the binary downloaded to the user's machine at package install time
* Add a fallback automatic from-source build for exotic OS/architectures

In [5]:
#Pkg.build("Rmath")

In [6]:
#Pkg.test("Rmath")

In [7]:
#using Rmath